In [ ]:
import tensorflow as tf
import pandas as pd
import re
import nltk

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

In [ ]:
df = pd.read_excel('Comments_all_bank.xlsx')

In [ ]:
def handle_mistype(sentiment):
    if not isinstance(sentiment, str):
        return sentiment
    pattern = r'(?i)\b(neu(tral)?|pos(itive)?|neg(ative)?)\w*\b'
    handled_sentiment = re.sub(pattern, lambda match: 'Neutral' if 'neu' in match.group(0).lower() else
                          ('Positive' if 'pos' in match.group(0).lower() else
                          ('Negative' if 'neg' in match.group(0).lower() else match.group(0))), sentiment)
    return handled_sentiment

df['Sentiment'] = df['Sentiment'].apply(handle_mistype)

In [ ]:
sentiment_mapping = {'Negative': 0, 'Neutral': 1 , 'Positive': 2 }

In [ ]:
df['Sentiment'] = df['Sentiment'].replace(sentiment_mapping)

In [ ]:
df['Sentiment'] = df['Sentiment'].replace({value: None for value in df['Sentiment'].unique()
                                           if value not in sentiment_mapping.values()})

In [ ]:
df = df.dropna(subset = ['Comment','Sentiment'])

In [ ]:
texts = df['Comment'].tolist()
labels = df['Sentiment']

In [ ]:
def cleaned_text(text):
    text = str(text)
    #remove symbols and special characters
    text = re.sub(r"^a-zA-Z0-9ğöəışçüĞÖƏIŞÇÜ\s", "", text)
    return text

cleaned_texts = [cleaned_text(text) for text in texts]

df['Comment'] = cleaned_texts
texts = df['Comment'].tolist()

In [ ]:
def cleaned_sentiment(label):
    label = str(label)
    label = re.sub(r"^0-9", "", label)
    return label
cleaned_labels = [int(cleaned_sentiment(label)) for label in labels]

df['Sentiment'] = cleaned_labels
labels = df['Sentiment']

In [ ]:
df.isnull().sum()

Comment      0
Sentiment    0
dtype: int64

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00


In [ ]:
from transformers import TFGPT2ForSequenceClassification , GPT2Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
labels = df['Sentiment'].tolist()

In [ ]:
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts , labels , test_size = 0.2 , random_state = 42)
val_texts, test_texts, val_labels , test_labels = train_test_split(temp_texts,temp_labels , test_size = 0.5, random_state = 42)

In [ ]:
train_inputs = tokenizer(train_texts, padding=True, truncation=True, return_tensors="tf")
val_inputs = tokenizer(val_texts, padding=True, truncation=True, return_tensors="tf")
test_inputs = tokenizer(test_texts, padding=True, truncation=True, return_tensors="tf")

In [ ]:
train_input_ids = train_inputs['input_ids']
train_attention_masks = train_inputs['attention_mask']
val_input_ids = val_inputs['input_ids']
val_attention_masks = val_inputs['attention_mask']
test_input_ids = test_inputs['input_ids']
test_attention_masks = test_inputs['attention_mask']
train_labels = tf.constant(train_labels)
val_labels = tf.constant(val_labels)
test_labels = tf.constant(test_labels)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_input_ids, train_attention_masks, train_labels)).shuffle(1000).batch(64)
val_dataset = tf.data.Dataset.from_tensor_slices((val_input_ids, val_attention_masks, val_labels)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((test_input_ids, test_attention_masks, test_labels)).batch(64)

In [ ]:
# model = TFGPT2ForSequenceClassification.from_pretrained("gpt2", num_labels = 3)

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10039341110625147071
xla_global_id: -1
]


In [ ]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification.from_pretrained("gpt2", num_labels = 3)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')],
                weighted_metrics = [])

All PyTorch model weights were used when initializing TFGPT2ForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFGPT2ForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5
179/179 [==============================] - 253s 970ms/step - loss: 0.0117 - accuracy: 0.9946 - val_loss: 3.4001e-04 - val_accuracy: 1.0000
Epoch 2/5
179/179 [==============================] - 119s 665ms/step - loss: 2.9500e-04 - accuracy: 1.0000 - val_loss: 1.3813e-04 - val_accuracy: 1.0000
Epoch 3/5
179/179 [==============================] - 120s 669ms/step - loss: 2.1097e-04 - accuracy: 1.0000 - val_loss: 7.9562e-05 - val_accuracy: 1.0000
Epoch 4/5
179/179 [==============================] - 119s 665ms/step - loss: 1.8569e-04 - accuracy: 1.0000 - val_loss: 5.0121e-05 - val_accuracy: 1.0000
Epoch 5/5
179/179 [==============================] - 119s 667ms/step - loss: 1.7066e-04 - accuracy: 1.0000 - val_loss: 3.1991e-05 - val_accuracy: 1.0000


In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

23/23 [==============================] - 29s 534ms/step - loss: 3.9247e-05 - accuracy: 1.0000
Test Loss: 3.924664270016365e-05
Test Accuracy: 1.0
